In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from CosinorPy import cosinor, file_parser

## Settings

In [2]:
sns.set_style("white")
colors = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02']
#flatui = ['#7fc97f','#beaed4','#fdc086','#ffff99','#386cb0','#f0027f']
sns.set_palette(colors)

# workday, weekend
workdays_colors=["#a3a3a3", "#426287"]

#good, bad
weather_colors = ["#5f5961", "#c25050"]


In [3]:
plt.rcParams.update({'font.size': 10})

## Functions

In [4]:
def plot_df_route_models(df, df_models, colors, plot_residuals=True, plot_phase = True, folder ="", x_label = '', y_label = ''):
    for row in df_models.iterrows():
        test = row[1].test
        n_components = row[1].n_components
        period = row[1].period
        X, Y = np.array(df[df.test == test].x), np.array(df[df.test == test].y)   
        
        route_id = int(test.split("_")[1])
        
        color = colors[route_id]
        
        
        if folder:
            cosinor.fit_me(X, Y, n_components = n_components, period = period, name = test, save_to = folder+'\\'+test+'_compnts='+str(n_components) +'_per=' + str(period), plot_residuals = plot_residuals, plot_phase = plot_phase, x_label = x_label, y_label = y_label, color=color)
        else:
            cosinor.fit_me(X, Y, n_components = n_components, period = period, name = test, save_to = "", plot_residuals = plot_residuals, plot_phase = plot_phase, x_label = x_label, y_label = y_label, color=color)


# Basic fits

In [5]:
df = file_parser.read_excel("data\\google_traffic.xlsx", remove_outliers=True)
df_results = cosinor.fit_group(df, n_components = [1,2,3,4,5,6,7,8,9,10], period=24, plot=False)
df_best_models = cosinor.get_best_models(df, df_results, n_components = [1,2,3,4,5,6,7,8,9,10])
plot_df_route_models(df, df_best_models, colors=colors, folder="cosinor_results\\figs", plot_phase=False, y_label="Pace [s/m]")
df_best_models.to_csv('cosinor_results\\best_models.csv', index=False)
df_best_models.to_excel('cosinor_results\\best_models.xlsx', index=False)

In [6]:
df_best_models.columns

Index(['test', 'period', 'n_components', 'p', 'q', 'p_reject', 'q_reject',
       'RSS', 'R2', 'R2_adj', 'log-likelihood', 'period(est)', 'amplitude',
       'acrophase', 'mesor', 'peaks', 'heights', 'troughs', 'heights2', 'ME',
       'resid_SE'],
      dtype='object')

In [7]:
df_summary = df_best_models[["test", "n_components", "p", "q", "peaks", "heights"]].copy()
df_summary["test"] = df_summary["test"].map(lambda x: x.split("_")[1])
df_summary["p"] = round(df_summary["p"], 2)
df_summary["q"] = round(df_summary["q"], 2)
df_summary["peaks"] = df_summary["peaks"].map(lambda x: [round(i,2) for i in x])
df_summary["heights"] = df_summary["heights"].map(lambda x: [round(i,3) for i in x])
df_summary.to_csv('cosinor_results\\best_models_summary.csv', index=False, sep="\t")

# Workdays

In [8]:
df = file_parser.read_excel("data\\google_traffic_workdays.xlsx", remove_outliers=True)
df_results = cosinor.fit_group(df, n_components = [1,2,3,4,5,6,7,8,9,10], period=24, plot=False)
df_best_models = cosinor.get_best_models(df, df_results, n_components = [1,2,3,4,5,6,7,8,9,10])
plot_df_route_models(df, df_best_models, colors=colors, folder="cosinor_results_workdays\\figs", plot_phase=False, y_label="Pace [s/m]")
df_best_models.to_csv('cosinor_results_workdays\\best_models_workdays.csv', index=False)
df_best_models.to_excel('cosinor_results_workdays\\best_models_workdays.xlsx', index=False)

In [9]:
df_best_models["route ID"] = df_best_models["test"].map(lambda x: x.split("_")[1])
df_best_models["day"] = df_best_models["test"].map(lambda x: x.split("_")[-1])


df_summary = df_best_models[["route ID", "day", "n_components", "p", "q", "peaks", "heights"]].copy()
df_summary["p"] = round(df_summary["p"], 2)
df_summary["q"] = round(df_summary["q"], 2)
df_summary["peaks"] = df_summary["peaks"].map(lambda x: [round(i,2) for i in x])
df_summary["heights"] = df_summary["heights"].map(lambda x: [round(i,3) for i in x])
df_summary.to_csv('cosinor_results_workdays\\best_models_workdays_summary.csv', index=False, sep="\t")

In [10]:
pairs = [("route_" + str(i) + "_workday", "route_" + str(i) + "_weekend") for i in range(6)] 
cosinor.plot_tuples_best_models(df, df_best_models, pairs, folder="cosinor_results_workdays\\figs_pairs", y_label="Pace [s/m]", colors = workdays_colors)#['red', 'black'])

In [11]:
df_best_models.columns

Index(['test', 'period', 'n_components', 'p', 'q', 'p_reject', 'q_reject',
       'RSS', 'R2', 'R2_adj', 'log-likelihood', 'period(est)', 'amplitude',
       'acrophase', 'mesor', 'peaks', 'heights', 'troughs', 'heights2', 'ME',
       'resid_SE', 'route ID', 'day'],
      dtype='object')

In [12]:
increase_work = {}

for work, weekends in pairs:
    route_id = int(work.split("_")[1])    
    #increase_work[route_id] = max(df_best_models.loc[df_best_models.test == work, "heights"]) / max(df_best_models.loc[df_best_models.test == weekends, "heights"])
    m1 = max(df_best_models.loc[df_best_models.test == work, "heights"].values[0])
    m2 = max(df_best_models.loc[df_best_models.test == weekends, "heights"].values[0])
    increase_work[route_id] = m1/m2

f = open('cosinor_results_workdays\\increase_work.csv', 'w')
f.write("route_id,workday/weekend\n")
for route_id, increase in increase_work.items():
    print(route_id, increase, file=f, sep=",")
f.close()
    

# Weather

In [13]:
df = file_parser.read_excel("data\\google_traffic_weather.xlsx", remove_outliers=True)
df_results = cosinor.fit_group(df, n_components = [1,2,3,4,5,6,7,8,9,10], period=24, plot=False)
df_best_models = cosinor.get_best_models(df, df_results, n_components = [1,2,3,4,5,6,7,8,9,10])
plot_df_route_models(df, df_best_models, colors=colors, folder="cosinor_results_weather\\figs", plot_phase=False, y_label="Pace [s/m]")
df_best_models.to_csv('cosinor_results_weather\\best_models_weather.csv', index=False)
df_best_models.to_excel('cosinor_results_weather\\best_models_weather.xlsx', index=False)

In [14]:
df_best_models

,test,period,n_components,p,q,p_reject,q_reject,RSS,R2,R2_adj,...,period(est),amplitude,acrophase,mesor,peaks,heights,troughs,heights2,ME,resid_SE
0,route_0_bad_weather,24,8,1.110223e-16,1.110223e-16,1.000000,1.0,0.051080,0.623726,0.618243,...,24,0.016231,-4.166777,0.122434,"[7.6076076076076085, 11.611611611611613, 15.91...","[0.13493213218597686, 0.12505558616607626, 0.1...","[2.5025025025025025, 5.205205205205205, 9.7097...","[0.10621131214616487, 0.11115999468025207, 0.1...",0.013383,0.006821
1,route_0_good_weather,24,9,1.110223e-16,1.110223e-16,0.999975,1.0,0.087783,0.563803,0.561051,...,24,0.011705,-4.009540,0.119683,"[7.507507507507508, 12.112112112112113, 15.315...","[0.12650568608328422, 0.12335166823632976, 0.1...","[2.5025025025025025, 5.005005005005005, 20.320...","[0.10797864098091518, 0.11196700740097452, 0.1...",0.010875,0.005546
2,route_1_bad_weather,24,8,1.110223e-16,1.110223e-16,1.000000,1.0,0.037681,0.635378,0.630150,...,3,0.012222,-24.319302,0.130595,"[7.6076076076076085, 11.611611611611613, 14.71...","[0.14121735131219854, 0.14281692163478818, 0.1...","[2.802802802802803, 4.904904904904905, 23.6236...","[0.11837412088485648, 0.12203404794391413, 0.1...",0.011401,0.005811
3,route_1_good_weather,24,10,1.110223e-16,1.110223e-16,0.999999,1.0,0.083838,0.640492,0.638011,...,3,0.012176,-24.738601,0.130306,"[7.6076076076076085, 11.811811811811813, 14.61...","[0.13631178998344456, 0.14248086781650793, 0.1...","[3.3033033033033035, 20.02002002002002, 23.123...","[0.11813445508214279, 0.13015140734073918, 0.1...",0.010546,0.005379
4,route_2_bad_weather,24,10,1.110223e-16,1.110223e-16,1.000000,1.0,0.227998,0.531781,0.523367,...,24,0.027687,-4.088159,0.153630,"[7.907907907907909, 10.910910910910912, 12.512...","[0.17763286709506543, 0.1568192722301633, 0.15...","[4.004004004004004, 9.70970970970971, 21.62162...","[0.12595517525176236, 0.1529156695346639, 0.13...",0.028083,0.014313
5,route_2_good_weather,24,9,1.110223e-16,1.110223e-16,1.000000,1.0,0.308463,0.591139,0.588575,...,24,0.022455,-4.088159,0.150217,"[7.7077077077077085, 11.111111111111112, 15.61...","[0.16568748668215336, 0.15581440086004184, 0.1...","[1.5015015015015016, 3.7037037037037037, 22.22...","[0.13082840450326289, 0.12776235905868347, 0.1...",0.020324,0.010365
6,route_3_bad_weather,24,6,1.110223e-16,1.110223e-16,1.000000,1.0,0.354108,0.522903,0.517755,...,24,0.032044,-4.245395,0.164843,"[11.911911911911913, 16.216216216216218]","[0.17313247384727287, 0.1968683705466351]","[0.4004004004004004, 4.504504504504505]","[0.13279938313024314, 0.136155218747369]",0.035014,0.017845
7,route_3_good_weather,24,7,1.110223e-16,1.110223e-16,1.000000,1.0,0.342010,0.684853,0.683311,...,24,0.023801,-4.088159,0.156592,"[11.511511511511513, 15.615615615615617]","[0.1760066095477007, 0.18039365228871426]",[],[],0.021438,0.010934
8,route_4_bad_weather,24,7,1.110223e-16,1.110223e-16,0.999903,1.0,0.233510,0.538902,0.533023,...,24,0.025010,-2.070285,0.142901,"[7.907907907907909, 11.711711711711713, 16.116...","[0.167910989286101, 0.1519980149849451, 0.1625...","[1.5015015015015016, 4.904904904904905]","[0.11789369652138423, 0.12100394546554621]",0.028614,0.014583
9,route_4_good_weather,24,9,1.110223e-16,1.110223e-16,0.996363,1.0,0.287914,0.611843,0.609397,...,24,0.019261,-4.166777,0.139617,"[7.307307307307307, 12.212212212212213, 15.915...","[0.15159069333511332, 0.15078319629750547, 0.1...","[0.7007007007007008, 2.1021021021021022, 4.704...","[0.12035693167695516, 0.1211820187599431, 0.12...",0.019684,0.010039


In [15]:
df_best_models["route ID"] = df_best_models["test"].map(lambda x: x.split("_")[1])
df_best_models["weather"] = df_best_models["test"].map(lambda x: x.split("_")[2])

df_summary = df_best_models[["route ID", "weather", "n_components", "p", "q", "peaks", "heights"]].copy()
df_summary["p"] = round(df_summary["p"], 2)
df_summary["q"] = round(df_summary["q"], 2)
df_summary["peaks"] = df_summary["peaks"].map(lambda x: [round(i,2) for i in x])
df_summary["heights"] = df_summary["heights"].map(lambda x: [round(i,3) for i in x])
df_summary.to_csv('cosinor_results_weather\\best_models_weather_summary.csv', index=False, sep="\t")

In [16]:
df_summary

,route ID,weather,n_components,p,q,peaks,heights
0,0,bad,8,0.0,0.0,"[7.61, 11.61, 15.92]","[0.135, 0.125, 0.139]"
1,0,good,9,0.0,0.0,"[7.51, 12.11, 15.32]","[0.127, 0.123, 0.131]"
2,1,bad,8,0.0,0.0,"[7.61, 11.61, 14.71, 18.02, 21.42]","[0.141, 0.143, 0.142, 0.139, 0.132]"
3,1,good,10,0.0,0.0,"[7.61, 11.81, 14.61, 21.22]","[0.136, 0.142, 0.142, 0.132]"
4,2,bad,10,0.0,0.0,"[7.91, 10.91, 12.51, 15.62]","[0.178, 0.157, 0.157, 0.181]"
5,2,good,9,0.0,0.0,"[7.71, 11.11, 15.62]","[0.166, 0.156, 0.173]"
6,3,bad,6,0.0,0.0,"[11.91, 16.22]","[0.173, 0.197]"
7,3,good,7,0.0,0.0,"[11.51, 15.62]","[0.176, 0.18]"
8,4,bad,7,0.0,0.0,"[7.91, 11.71, 16.12]","[0.168, 0.152, 0.163]"
9,4,good,9,0.0,0.0,"[7.31, 12.21, 15.92]","[0.152, 0.151, 0.159]"


In [17]:
pairs = [("route_" + str(i) + "_good_weather", "route_" + str(i) + "_bad_weather") for i in range(6)] 
cosinor.plot_tuples_best_models(df, df_best_models, pairs, folder="cosinor_results_weather\\figs_pairs", y_label="Pace [s/m]", colors = weather_colors)#['black', 'red'])

In [18]:
increase_weather = {}

for good, bad in pairs:
    route_id = int(good.split("_")[1])    
    #increase_work[route_id] = max(df_best_models.loc[df_best_models.test == work, "heights"]) / max(df_best_models.loc[df_best_models.test == weekends, "heights"])
    m1 = max(df_best_models.loc[df_best_models.test == bad, "heights"].values[0])
    m2 = max(df_best_models.loc[df_best_models.test == good, "heights"].values[0])
    increase_weather[route_id] = m1/m2

f = open('cosinor_results_weather\\increase_weather.csv', 'w')
f.write("route_id,bad/good_weather\n")
for route_id, increase in increase_weather.items():
    print(route_id, increase, file=f, sep=",")
f.close()